In [8]:
import docx
import re
import bs4

In [11]:
# Tworzenie bazy danych z normami

with open("aktualne.html", encoding="utf-8") as website:
    soup = bs4.BeautifulSoup(website, "html.parser")
    
lista = []
for match in soup.find_all("span", class_="highlighted-search-term"):
    lista.append(match.next_sibling)

for s in lista:
    s = "PN"+s

blocks = soup.find_all("div", class_="p4s-search-results-values")    

db = []
for b in blocks:
    tytul = None
    numer = None
    zastepuje = []
    
    if len(b.find_all("span", string=re.compile("Tytuł normy"))) == 0:
        tytul = ""
    else:
        tytul = str(b.find_all("span", string=re.compile("Tytuł normy"))[0].find_next("a").string).replace('\t', '').replace('\n', '')
        numer = "PN"+ str(b.find_all("span", class_="highlighted-search-term")[0].next_sibling).replace('\t', '').replace('\n', '')
        
        if len(b.find_all(string=re.compile("Zastępuje"))) != 0:
            zastapione_normy = b.find_all(string=re.compile("Zastępuje"))[0].parent.parent.next_sibling.find_next("ul").find_all("a")
            for z in zastapione_normy:
                z = z.contents[0].replace('\t', '').replace('\n', '')
                zastepuje.append(z)
            
#             zastepuje = str(b.find_all(string=re.compile("Zastępuje"))[0].parent.parent.next_sibling.find_next("a").contents[0]).replace('\t', '').replace('\n', '')
        
    db.append((tytul, numer, zastepuje))

In [12]:
# Przeszukiwanie dokumentu specyfikacji

specyfikacja = docx.Document("test_data/specyfikacja.docx")

# pętla for po paragrafach
content = ""
for p in specyfikacja.paragraphs:
    content += p.text

# specyfikacja.save("test_data/kopia.docx")

regex = r"PN(?: |-).{1,30}?(?:(?::\d{4})(?:-\d\d|))(?:[\S]+?(?:\d{4})|)(?:-\d{2}|)"
normy = re.findall(regex, content)

In [14]:
# Porównanie wyników wyszukiwania z bazą

results = []

for n in normy:
    mark = "Not found in db"
    state = "Unknown" # Up-to-date
    newest = None
    
    for d in db:
        if n == d[1]:
            mark = "Found in db"
            state = "Up-to-date"
            break
            
        else:
            for old in d[2]:
                if n == old:
                    mark = "Found in db"
                    state = "Out-of-date"
                    newest = d[1]
                    break
    results.append((n, mark, state, newest))

In [15]:
results

[('PN-B-20130:1999', 'Not found in db', 'Unknown', None),
 ('PN-EN 14891:2012', 'Not found in db', 'Unknown', None),
 ('PN-EN 12004:2002', 'Not found in db', 'Unknown', None),
 ('PN-EN 2004:2002', 'Not found in db', 'Unknown', None),
 ('PN-EN 14411:2005', 'Not found in db', 'Unknown', None),
 ('PN-EN 12004:2002', 'Not found in db', 'Unknown', None),
 ('PN-EN 14411:2005', 'Not found in db', 'Unknown', None),
 ('PN-EN 2004:2002', 'Not found in db', 'Unknown', None),
 ('PN-EN 12859:2002', 'Not found in db', 'Unknown', None),
 ('PN-EN 12860:2002', 'Found in db', 'Up-to-date', None),
 ('PN-B-79405:1997/Ap1:1999', 'Not found in db', 'Unknown', None),
 ('PN-B-30042:1997', 'Not found in db', 'Unknown', None),
 ('PN-EN 14411:2005', 'Not found in db', 'Unknown', None),
 ('PN-EN 13964:2004', 'Not found in db', 'Unknown', None),
 ('PN-EN 22768-1:1999', 'Not found in db', 'Unknown', None),
 ('PN-B-02151-2:2018-01', 'Found in db', 'Up-to-date', None),
 ('PN-B-10085:2001', 'Not found in db', 'Unknown

# Powiększenie bazy

Trzeba powiększyć bazę o sektory: chemia, elektryka
Oraz o numery starych norm ze spisu

In [16]:
# Eksport danych do nowego pliku

# for paragraf in specyfikacja.paragraphs:
#     for i in range(len(results)):
#         if results[i][3] != None:             
#             paragraf.text = paragraf.text.replace(results[i][0], results[i][3], 1)            
#     print(paragraf.text)

# specyfikacja.save("test_data/output.docx")    